In [12]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from guaranteed_control.nn_reachability.nn_reachability_tf import nn_interval_py
from guaranteed_control.intervals.interval import Interval
import tensorflow.keras as keras
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
def sample_points(interval, N=5000):

    low, high = interval.high_low()
    points = []

    for i in range (len(low)):
        points.append(np.random.uniform(low[i], high[i], N).reshape(-1, 1))

    return np.concatenate(points, axis=1)

In [13]:
model = keras.models.load_model("./models/mpc.tf")
test_interval = Interval(interval=[[0, 0.2], [0, 0.2]])
points = sample_points(test_interval)

In [29]:
def cut_interval(interval, epsilon=0.01):

    intervals = [interval]
    outputs = []

    while len(intervals) != 0:
        interval = intervals.pop()

        if interval.length() > epsilon:
            interval1, interval2 = interval.bissection()
            intervals.append(interval1)
            intervals.append(interval2)
        else:
            outputs.append(Interval(interval=[np.concatenate(nn_interval_py(model, interval.intervals), axis=0)]).intervals[0])
    
    return outputs

In [10]:
output_interval = Interval(interval=[np.concatenate(nn_interval_py(model, test_interval.intervals), axis=0)])
output_interval.intervals

array([[0.03038868, 0.03945816]], dtype=float32)

In [11]:
nn_points = sample_points(output_interval)
scipy.io.savemat('lyap_arrays.mat', {'nn_points': nn_points, 'nn_interval': output_interval.intervals})

In [30]:
outputs = cut_interval(test_interval)

In [31]:
scipy.io.savemat('lyap_arrays.mat', {'nn_intervals': outputs})

In [32]:
outputs[1]

array([-0.46978796, -0.451783  ], dtype=float32)